# 2.

data prep for ML training. requires ```db_data_scraper.ipynb```

In [2]:
import sqlite3
import pandas as pd
import datetime
import numpy as np
from matplotlib import pyplot as plt
import time

database = "../main.db"

In [3]:
# conn.close() # comment this if running the first time
conn = sqlite3.connect(database)

## get data from the db

select only those combinations (ticker,earnings_date) where the EPS actual was positive.

In [4]:
q = """
    SELECT
        *
    FROM yf_secondary_expanded
    WHERE eps_diff > 0.0
"""
basic=pd.read_sql_query(q,conn).set_index(['ticker','earnings_date'])

In [5]:
basic

financial_date  rnk  eps_est  eps_actual  eps_diff  \
ticker earnings_date                                                      
A      2021-05-25        2020-10-31    1     0.83        0.97      0.14   
       2021-08-17        2020-10-31    1     0.99        1.10      0.11   
       2021-11-22        2021-10-31    1     1.18        1.21      0.03   
       2022-02-22        2021-10-31    1     1.18        1.21      0.03   
       2022-05-24        2021-10-31    1     1.12        1.13      0.01   
...                             ...  ...      ...         ...       ...   
ZTS    2022-05-05        2021-12-31    1     1.23        1.32      0.09   
ZUO    2021-12-01        2021-01-31    1    -0.03       -0.02      0.01   
       2022-03-02        2022-01-31    1    -0.02       -0.01      0.01   
       2022-08-24        2022-01-31    1    -0.05       -0.03      0.02   
       2022-12-06        2022-01-31    1    -0.06       -0.02      0.04   

                      eps_diff_pct    tot_assets  common_stock       tot_cap  \
ticker earnings_date                                                           
A      2021-05-25           0.1729  9.627000e+09     3000000.0  7.157000e+09   
       2021-08-17           0.1089  9.627000e+09     3000000.0  7.157000e+09   
       2021-11-22           0.0298  1.070500e+10     3000000.0  8.118000e+09   
       2022-02-22           0.0289  1.070500e+10     3000000.0  8.118000e+09   
       2022-05-24           0.0089  1.070500e+10     3000000.0  8.118000e+09   
...                            ...           ...           ...           ...   
ZTS    2022-05-05           0.0706  1.390000e+10     5000000.0  1.113500e+10   
ZUO    2021-12-01           0.3103  4.215070e+08       12000.0  1.735790e+08   
       2022-03-02           0.5833  4.412520e+08       13000.0  1.706070e+08   
       2022-08-24           0.4000  4.412520e+08       13000.0  1.706070e+08   
       2022-12-06           0.6552  4.412520e+08       13000.0  1.706070e+08   

                          tot_debt  ...  expense_to_income  assets_to_profit  \
ticker earnings_date                ...                                        
A      2021-05-25     2.359000e+09  ...           2.353428          3.393373   
       2021-08-17     2.359000e+09  ...           2.353428          3.393373   
       2021-11-22     2.729000e+09  ...           1.529324          3.142060   
       2022-02-22     2.729000e+09  ...           1.529324          3.142060   
       2022-05-24     2.729000e+09  ...           1.529324          3.142060   
...                            ...  ...                ...               ...   
ZTS    2022-05-05     6.743000e+09  ...           0.952551          2.539741   
ZUO    2021-12-01     6.928300e+07  ...          -3.364545          2.413438   
       2022-03-02     5.875500e+07  ...          -3.148478          2.135449   
       2022-08-24     5.875500e+07  ...          -3.148478          2.135449   
       2022-12-06     5.875500e+07  ...          -3.148478          2.135449   

                      debt_to_debt  debt_to_cap  profit_to_reve  \
ticker earnings_date                                              
A      2021-05-25         0.389148     0.329607        0.531373   
       2021-08-17         0.389148     0.329607        0.531373   
       2021-11-22         0.456211     0.336167        0.539168   
       2022-02-22         0.456211     0.336167        0.539168   
       2022-05-24         0.456211     0.336167        0.539168   
...                            ...          ...             ...   
ZTS    2022-05-05         0.461219     0.605568        0.703832   
ZUO    2021-12-01              NaN     0.399144        0.571836   
       2022-03-02              NaN     0.344388        0.595931   
       2022-08-24              NaN     0.344388        0.595931   
       2022-12-06              NaN     0.344388        0.595931   

                      on_earnings_close  on_earnings_volume    market_cap  \
ticker earning

In [6]:
hist_q = """
    SELECT 
        ticker,
        DATE(trade_date) trade_date,
        close
    FROM yf_history 
"""
hist  = pd.read_sql_query(hist_q,conn)

In [7]:
hist

,ticker,trade_date,close
0,A,2019-01-31,76.050003
1,A,2019-02-01,76.040001
2,A,2019-02-04,76.029999
3,A,2019-02-05,76.309998
4,A,2019-02-06,76.300003
...,...,...,...
1020411,KBH,2023-01-13,35.099998
1020412,KBH,2023-01-17,35.189999
1020413,KBH,2023-01-18,35.290001
1020414,KBH,2023-01-19,34.799999


In [24]:
def get_history_for(ticker, earnings_date):
    sub_basic = basic.loc[(ticker, earnings_date)]
    sub_start = sub_basic['history_start_date'][0]
    sub_end   = sub_basic['history_end_date'][0]
    sub_hist  = hist[(hist.ticker==ticker) & (hist.trade_date>=sub_start) & (hist.trade_date<=sub_end)].sort_values('trade_date', ascending=True)
    if sub_hist.shape[0]==0:
        return -1,-1
    init_close = sub_hist.head(1)['close'].values[0]
    sub_hist['price_pct'] = sub_hist.close/init_close-1.0
    return sub_hist['price_pct'].min(), sub_hist['price_pct'].max()

# price_min, price_max = get_history_for('ZUO', '2022-12-06')

In [25]:
i = 0
for ticker, earnings_date in basic.index[2403:]:
    print(i, ticker, earnings_date)
    price_min, price_max = get_history_for(ticker, earnings_date)
    basic.loc[(ticker, earnings_date),'price_min'] = price_min
    basic.loc[(ticker, earnings_date),'price_max'] = price_max
    i+=1

0 INDB 2023-01-20
1 INGR 2021-08-03
2 INGR 2021-11-02
3 INGR 2022-05-05
4 INGR 2022-08-09
5 INGR 2022-11-03
6 INTA 2021-11-10
7 INTA 2022-09-07
8 INTA 2022-11-07
9 INTC 2021-07-22
10 INTC 2021-10-21
11 INTC 2022-01-26
12 INTC 2022-04-28
13 INTC 2022-10-27
14 INTU 2021-08-24
15 INTU 2021-11-18
16 INTU 2022-05-24
17 INTU 2022-08-23
18 INTU 2022-11-29
19 INVH 2021-07-28
20 INVH 2021-10-27
21 INVH 2022-04-27
22 INVH 2022-07-27
23 IOBT 2022-08-11
24 IOBT 2022-11-09
25 IONQ 2022-03-28
26 IONQ 2022-05-16
27 IONQ 2022-08-15
28 IONS 2021-08-04
29 IONS 2022-02-24
30 IONS 2022-05-04
31 IONS 2022-11-09
32 IOT 2022-06-02
33 IOT 2022-08-31
34 IOT 2022-12-01
35 IP 2022-04-28
36 IP 2022-07-28
37 IPG 2021-07-21
38 IPG 2021-10-21
39 IPG 2022-02-10
40 IPG 2022-04-28
41 IPG 2022-07-21
42 IPG 2022-10-21
43 IQ 2021-05-18
44 IQ 2021-08-12
45 IQ 2021-11-17
46 IQ 2022-05-26
47 IQ 2022-08-30
48 IQV 2021-04-22
49 IQV 2021-07-27
50 IQV 2021-10-21
51 IQV 2022-02-15
52 IQV 2022-04-27
53 IQV 2022-07-21
54 IQV 2022-1

In [29]:
basic.price_min.isnull().sum()

0

In [30]:
basic.to_csv('raw_features.csv')